In [1]:
from DataReader import DataReaderChallangePolimi2017
import implicit
from CB_asy import CB_asymmetric_cosine
from CF_IB_asy import CF_IB_asymmetric_cosine
from CF_UB_asy import CF_UB_asymmetric_cosine
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,TFIDFRecommender)
from sklearn.preprocessing import normalize
import numpy as np
import scipy.sparse as sp
import pickle

evaluation = True
rebuild = False
dr = DataReaderChallangePolimi2017(evaluation=evaluation,rebuild=rebuild)

urm = dr.getURM_csr()
t_p = dr.target_playlists
t_t = dr.target_tracks
if evaluation:
    filesslim = "./DumpData/sim_bpr_10e_100k2.npz" # or "./DumpData/sim_bpr_10e_100k.npz"
else:
    filesslim = "./DumpData/sim_bpr_10e_100k_fulltrain.npz"

Date Reader Challange Polimi 2017
Loading data from ./DumpData/DataReaderTrainTestDump
Dataset train-test loaded


In [31]:
print dr.train
print dr.artists

[[ 8890 22426 54743 ..., 32763 32763 32763]
 [71058 19741 71298 ..., 87949 62773 70493]]
[[77187 69189 37833 ..., 64702 57834 68922]
 [    2     4     2 ...,   980 14291 17533]]
[[ 8890 71058]
 [22426 19741]
 [54743 71298]
 ..., 
 [32763 87949]
 [32763 62773]
 [32763 70493]]


In [98]:
dic_ar = {}
ar_tr = dr.artists.T
for row in ar_tr:
    dic_ar[row[0]] = row[1]

n_playlists = dr.n_playlists
n_attributes = dr.n_attributes

print n_attributes
print n_playlists

train = dr.train.T
r,c,v = [],[],[]
for row in train:
    r.append(row[0])
    c.append(dic_ar[row[1]])
    v.append(1)

icm_ar = sp.coo_matrix((v, (r, c)), shape=(n_playlists, n_attributes),dtype=np.float).tocsr()

77040
57561


In [96]:
#playlist-artist
model = BM25Recommender(K=55)
model.fit(icm_ar)
s = model.similarity
rt = s*urm
rt = normalize(rt, axis=0)
if evaluation: print dr.evaluateMAP(rt,verbose=False)
rt = dr.reduceRM(rt)

0.0581924444444


In [97]:
dic_al = {}
al_tr = dr.albums.T
for row in al_tr:
    dic_al[row[0]] = row[1]
    
n_playlists = dr.n_playlists
n_attributes = dr.n_attributes

print n_attributes
print n_playlists

train = dr.train.T
r,c,v = [],[],[]
for row in train:
    if row[1] not in dic_al: continue
    r.append(row[0])
    c.append(dic_al[row[1]])
    v.append(1)

icm_al = sp.coo_matrix((v, (r, c)), shape=(n_playlists, n_attributes),dtype=np.float).tocsr()

77040
57561


In [109]:
#playtlist-album
model = BM25Recommender(K=100)
model.fit(icm_al)
s = model.similarity
rt = s*urm
rt = normalize(rt, axis=0)
if evaluation: print dr.evaluateMAP(rt,verbose=False)
rt = dr.reduceRM(rt)

0.0704124722222


In [107]:
r,c,v = [],[],[]

for row in train:
    r.append(row[0])
    c.append(dic_ar[row[1]])
    v.append(1)

for row in train:
    if row[1] not in dic_al: continue
    r.append(row[0])
    c.append(dic_al[row[1]])
    v.append(1)

icm_al_ar = sp.coo_matrix((v, (r, c)), shape=(n_playlists, n_attributes),dtype=np.float).tocsr()

In [108]:
model = BM25Recommender(K=80)
model.fit(icm_al_ar)
s = model.similarity
rt = s*urm
rt = normalize(rt, axis=0)
if evaluation: print dr.evaluateMAP(rt,verbose=False)
rt = dr.reduceRM(rt)

0.0716002777778
